# 导入模块

In [1]:
import pandas as pd
import shutil
import os
import re
import pymysql
import warnings
import datetime
import time
import openpyxl
from openpyxl.styles import Font, Alignment, Side, Border, PatternFill, GradientFill, PatternFill
from openpyxl.formatting.rule import DataBarRule
from openpyxl import load_workbook
from openpyxl import Workbook
warnings.filterwarnings('ignore')

# 连接数据库

In [2]:
conn1 = pymysql.connect(host = 'am-wz9el267w54i2r7ip131930o.ads.aliyuncs.com' # 连接名称，默认127.0.0.1
,user = 'yjy_user' # 用户名
,passwd='Yjy123456' # 密码
,port= 3306 # 端口，默认为3306
,db='yun-campusjob'#数据库名称
,charset='utf8' # 字符编码
)

conn2 = pymysql.connect(host = 'am-wz9el267w54i2r7ip131930o.ads.aliyuncs.com' # 连接名称，默认127.0.0.1
,user = 'yjy_user' # 用户名
,passwd='Yjy123456' # 密码
,port= 3306 # 端口，默认为3306
,db='warehouse'#数据库名称
,charset='utf8' # 字符编码
)

# 参数

In [3]:
#格式定义
font2 = Font(bold=True,size=10,name='微软雅黑')
font1 = Font(bold=False,size=10,name='微软雅黑')
fill = PatternFill('solid',fgColor='ddebf7')#颜色
bar_format = DataBarRule(start_type="percentile", start_value=0, end_type="percentile", end_value=100, color='638EC6')
side=Side(style='thin', color='000000')
side1=Side(style='medium', color='008000')
border=Border(left=side,right=side,top=side,bottom=side)
border1=Border(left=side,right=side,top=side,bottom=side1)
border2=Border(left=side,right=side,top=side1,bottom=side)

# 定义函数

In [4]:
def make_sum_table(isOne,school_data,sheet1,query_name,query_name1,company_query_num,curr_time):
    line_num = school_data.shape[0]
    data_values = school_data.values
    sheet1.column_dimensions['C'].width = 25#宽度
    sheet1.column_dimensions['K'].width = 14.5#宽度
    sheet1.column_dimensions['J'].width = 14.5#宽度
    sheet1.merge_cells('B2:G7')
    sheet1.merge_cells('B9:G9')
    sheet1.merge_cells('I9:O9')
    sheet1['B9'] = '分学院回收率统计表'
    sheet1['B9'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
    sheet1['I9'] = '分学院分专业回收率统计表'
    sheet1['I9'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
    sheet1['G10'] = '未提交'
    sheet1['B10'] = '学历'
    sheet1['C10'] = '学院'
    sheet1['D10'] = '生源人数'
    sheet1['E10'] = '已提交人数'
    sheet1['F10'] = '回收率'
    sheet1['O10'] = '未提交'
    sheet1['I10'] = '学历'
    sheet1['J10'] = '学院'
    sheet1['K10'] = '专业'
    sheet1['L10'] = '生源人数'
    sheet1['M10'] = '已提交人数'
    sheet1['N10'] = '回收率'
    for index in {'G10','B10','C10','D10','E10','F10','O10','I10','J10','K10','L10','M10','N10','G9','B9','C9','D9','E9','F9','O9','I9','J9','K9','L9','M9','N9'}:
        sheet1[index].font = font2
        sheet1[index].fill = fill
        sheet1[index].border = border
    i = 0
    j = 0
    left_list = []
    temp = 0
    temp_id = 11
    while i<line_num:
        if data_values[i][2] != '-':
            for col_num in range(ord('I'),ord('P')):
                sheet1[f'{chr(col_num)}{i-j+11}'] = data_values[i][0+col_num-ord('I')]
                sheet1[f'{chr(col_num)}{i-j+11}'].font = font1
                sheet1[f'{chr(col_num)}{i-j+11}'].border = border
        else:
            j = j + 1
            if data_values[i][1] != '-':
                left_list.append([i,1])
            else:
                left_list.append([i,0])
        i = i + 1
    sheet1.conditional_formatting.add("N11:N" + str(i-j+10), bar_format)# 设置数据条
    i = 1
    while i<len(left_list):
        if left_list[i][1]==1:
            sheet1[f'C{i+9}'] = data_values[left_list[i][0]][1]
            sheet1[f'C{i+9}'].font = font1
            sheet1[f'C{i+9}'].border = border
            sheet1[f'D{i+9}'] = data_values[left_list[i][0]][3]
            sheet1[f'D{i+9}'].font = font1
            sheet1[f'D{i+9}'].border = border
            sheet1[f'E{i+9}'] = data_values[left_list[i][0]][4]
            sheet1[f'E{i+9}'].font = font1
            sheet1[f'E{i+9}'].border = border
            sheet1[f'F{i+9}'] = data_values[left_list[i][0]][5]
            sheet1[f'F{i+9}'].font = font1
            sheet1[f'F{i+9}'].border = border
            sheet1[f'G{i+9}'] = data_values[left_list[i][0]][6]
            sheet1[f'G{i+9}'].font = font1
            sheet1[f'G{i+9}'].border = border
        else:
            if temp==0:
                temp = left_list[i][0]
            else:
                sheet1.merge_cells(f'B{i+9}:C{i+9}')
                sheet1[f'B{i+9}'] = f'{data_values[temp][0]} 汇总'
                sheet1[f'B{i+9}'].font = font2
                sheet1[f'B{i+9}'].border = border
                sheet1[f'C{i+9}'].border = border
                sheet1[f'B{i+9}'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
                sheet1[f'D{i+9}'] = data_values[temp][3]
                sheet1[f'D{i+9}'].font = font1
                sheet1[f'D{i+9}'].border = border
                sheet1[f'E{i+9}'] = data_values[temp][4]
                sheet1[f'E{i+9}'].font = font1
                sheet1[f'E{i+9}'].border = border
                sheet1[f'F{i+9}'] = data_values[temp][5]
                sheet1[f'F{i+9}'].font = font1
                sheet1[f'F{i+9}'].border = border
                sheet1[f'G{i+9}'] = data_values[temp][6]
                sheet1[f'G{i+9}'].font = font1
                sheet1[f'G{i+9}'].border = border
                sheet1.merge_cells(f'B{temp_id}:B{i+8}')
                sheet1[f'B{temp_id}'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
                sheet1[f'B{temp_id}'] = data_values[temp][0]
                sheet1[f'B{temp_id}'].font = font1
                while temp_id<i+9:
                    sheet1[f'B{temp_id}'].border = border
                    temp_id=temp_id+1
                temp = left_list[i][0]
        i = i + 1
    sheet1.merge_cells(f'B{i+9}:C{i+9}')
    sheet1[f'B{i+9}'] = f'{data_values[temp][0]} 汇总'
    sheet1[f'B{i+9}'].font = font2
    sheet1[f'B{i+9}'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
    sheet1[f'B{i+9}'].border = border
    sheet1[f'C{i+9}'].border = border
    sheet1[f'D{i+9}'] = data_values[temp][3]
    sheet1[f'D{i+9}'].font = font1
    sheet1[f'D{i+9}'].border = border
    sheet1[f'E{i+9}'] = data_values[temp][4]
    sheet1[f'E{i+9}'].font = font1
    sheet1[f'E{i+9}'].border = border
    sheet1[f'F{i+9}'] = data_values[temp][5]
    sheet1[f'F{i+9}'].font = font1
    sheet1[f'F{i+9}'].border = border
    sheet1[f'G{i+9}'] = data_values[temp][6]
    sheet1[f'G{i+9}'].font = font1
    sheet1[f'G{i+9}'].border = border
    try:
        sheet1.merge_cells(f'B{temp_id+1}:B{i+8}')
    except:
        pass
    sheet1[f'B{temp_id+1}'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
    sheet1[f'B{temp_id+1}'] = data_values[temp][0]
    sheet1[f'B{temp_id+1}'].font = font1
    while temp_id<i+9:
        sheet1[f'B{temp_id}'].border = border
        temp_id=temp_id+1
    if len(school_data['xl'].unique()) > 2:
        i = i + 1
        temp = 0
        sheet1.merge_cells(f'B{i+9}:C{i+9}')
        sheet1[f'B{i+9}'] = '全体毕业生'
        sheet1[f'B{i+9}'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
        sheet1[f'B{i+9}'].font = font2
        sheet1[f'B{i+9}'].border = border
        sheet1[f'C{i+9}'].border = border
        sheet1[f'D{i+9}'] = data_values[temp][3]
        sheet1[f'D{i+9}'].font = font2
        sheet1[f'D{i+9}'].border = border
        sheet1[f'E{i+9}'] = data_values[temp][4]
        sheet1[f'E{i+9}'].font = font2
        sheet1[f'E{i+9}'].border = border
        sheet1[f'F{i+9}'] = data_values[temp][5]
        sheet1[f'F{i+9}'].font = font2
        sheet1[f'F{i+9}'].border = border
        sheet1[f'G{i+9}'] = data_values[temp][6]
        sheet1[f'G{i+9}'].font = font2
        sheet1[f'G{i+9}'].border = border
    else:
        merge_data_temp = sheet1['B12'].value
        try:
            sheet1.merge_cells(f'B11:B{i+7}')
        except:
            pass
        sheet1['B11'] = merge_data_temp
        sheet1['B11'].font = font1
        sheet1['B11'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
    sheet1.conditional_formatting.add(f"F11:F{i+9}", bar_format)# 设置数据条
    
    time_str1 = curr_time.strftime("%Y年%m月%d日 %H：%M")
    if (isOne==True):
        text = ''
    else:
        print(isOne)
        text = f'{isOne}届'
    if len(company_query_num)>0:
        sheet1['B2'] = f'''【调研进度汇报】截至时间：{time_str1}
单位问卷《{query_name1}》回收数量{company_query_num[0]}份；
学生问卷《{query_name}》{text}回收数量{data_values[temp][4]}份，回收率{data_values[temp][5]}%，未提交数量{data_values[temp][6]}人。（生源信息来自云就业）
下表为各学院及专业回收率统计表。'''
    else:
        sheet1['B2'] = f'''【调研进度汇报】截至时间：{time_str1}
学生问卷《{query_name}》{text}回收数量{data_values[temp][4]}份，回收率{data_values[temp][5]}%，未提交数量{data_values[temp][6]}人。（生源信息来自云就业）
下表为各学院及专业回收率统计表。'''
    
    sheet1['B2'].alignment=Alignment(horizontal="left", vertical="center", text_rotation=0,wrapText=True)
    sheet1['B2'].font = font1
    sheet1['B2'].fill = fill
    return

In [5]:
def make_detail_table(data_no_students,sheet2):
    i = 1
    while i<data_no_students.shape[0]+2:
        if i == 1:
            sheet2['A1'].alignment=Alignment(horizontal="center", vertical="center", text_rotation=0)
            for col_num in range(ord('A'),ord('H')):
                sheet2[f'{chr(col_num)}{i}'].font = font2
                sheet2[f'{chr(col_num)}{i}'].border = border1
        else:
            for col_num in range(ord('A'),ord('H')):
                sheet2[f'{chr(col_num)}{i}'].font = font1
            sheet2[f'A{i}'] = i-1
            if i !=2:
                for col_num in range(ord('A'),ord('H')):
                    sheet2[f'{chr(col_num)}{i}'].border = border
            else:
                for col_num in range(ord('A'),ord('H')):
                    sheet2[f'{chr(col_num)}{i}'].border = border2
        i = i + 1
    sheet2.column_dimensions['D'].width = 20#宽度
    sheet2.column_dimensions['A'].width = 4.5#宽度
    sheet2.column_dimensions['E'].width = 17#宽度
    sheet2.column_dimensions['F'].width = 17#宽度
    sheet2.column_dimensions['G'].width = 17#宽度
    return

# 运行

In [6]:
q_id_group=(7191,7505)
curr_time = datetime.datetime.now()
time_str = curr_time.strftime('%Y%m%d')
def temp_f(x):
    return f"SELECT a.school_id,school_name,open_url,name,q_define_id,for_graduate_year FROM `sch_q_define` a join sch_school b on a.school_id=b.school_id WHERE `q_define_id` in {str(q_id_group)} and type = {x}  "#0是学生
data_student_survey = pd.read_sql(con=conn1,sql=temp_f(0))
data_company_survey = pd.read_sql(con=conn1,sql=temp_f(1))

data_quest_url = re.sub('\[|\]','',str(data_student_survey['open_url'].tolist()))
data_school_id = re.sub('\[|\]','',str(data_student_survey['school_id'].tolist()))
data_school_name = re.sub('\[|\]','',str(data_student_survey['school_name'].tolist()))
year_list = re.sub("\'|\'|\[|\]",'',str(data_student_survey['for_graduate_year'].tolist()))

In [7]:
if not data_company_survey.empty:
    data_com_url = re.sub('\[|\]','',str(data_company_survey['open_url'].tolist()))

#剔除学生名单
try:
    ori_data = pd.read_excel(r'E:/桌面/数据excel/survey_list_2022_需剔除学生.xlsx')
    data_kaichu_school = re.sub('\[|\]','',str(ori_data['school_name'].tolist()))
    data_kaichu_graduate_year = re.sub('\[|\]','',str(ori_data['year'].tolist()))
    data_kaichu_xh = re.sub('\[|\]','',str(ori_data['xh'].tolist()))
except:
    data_kaichu_school=''
    data_kaichu_graduate_year=''
    data_kaichu_xh=''
    
if not data_company_survey.empty:
    sql = f'''
    SELECT school_name,num from
    (
    select b.school_id,COUNT(a.yb_id) as num FROM 
    c_survey_yb a 
    INNER JOIN
    (SELECT a.school_id,a.yb_id #根据q_define_id拿到所有样本id
    FROM `c_survey_result` a
    WHERE a.q_define_id in( SELECT DISTINCT q_define_id FROM sch_q_define WHERE open_url IN ({data_com_url}))
    GROUP BY a.school_id,a.yb_id
    )b ON a.yb_id = b.yb_id
    where a.is_submit = 1
    GROUP BY b.school_id)a
    join sch_school b
    on a.school_id= b.school_id
    '''

    data_company_num = pd.read_sql(con=conn1,sql=sql)

sql = f"""
SELECT school_name AS `学校名称`,case  when c.xl in ('本科生毕业','本科生结业','本科','本科毕业生','普通本科','本科毕业','本科一批','本1','本科三批','本科生在读','本科四年','本科生学历','专升本','本科生','二学位毕业') then '本科毕业生' 
         when c.xl in ('硕士生毕业','硕士研究生','研究生','研究生毕业','硕士毕业','全日制硕士研究生','硕士研究生毕业','全日制学术学位硕士','全日制专业学位硕士','硕士生','非全日制硕士研究生','非全日制专业学位硕士','硕士生结业','硕士毕业生','博士生毕业','博士研究生','博士毕业','博士','博士研究生毕业','全日制博士研究生','全日制学术学位博士','博士毕业生','博士生','博士生结业','硕士研究生  （非全','在读硕士研究生') then '毕业研究生'
         when c.xl in ('专科生毕业','专科','大专','专科毕业生','专科生','大学专科','专科毕业','专科生结业','普通大专','大专在读') then '专科毕业生' else null end as xl
,graduate_year,szyx AS `院系`, c.zy AS '专业',
(CASE WHEN sum(txrs) IS NULL THEN 0 ELSE sum(txrs) END) AS `填写人数`,sum(zyrs)-(CASE WHEN sum(txrs) IS NULL THEN 0 ELSE sum(txrs) END) AS '未填写人数',sum(zyrs) AS `总人数`, round((CASE WHEN sum(txrs) IS NULL THEN 0 ELSE sum(txrs) END)/sum(zyrs)*100,2) AS `回收率`
FROM #每一个学生的信息，答了问卷的有txrs
(SELECT school_id,year,xh,sfzhm,count(yb_id) AS txrs #txrs用于sum计数
FROM (SELECT a.yb_id,a.school_id,a.xm,a.x AS yx,a.sfzhm,a.xh,a.bjhm AS bj,a.year,if(a.is_male=1,'男','女') AS xb #根据样本id拿到回答的学生信息
FROM 
sch_survey_yb a 
INNER JOIN
(SELECT a.q_quest_id,a.yb_id #根据q_define_id拿到所有样本id
FROM `sch_survey_result` a
WHERE a.is_yb = 1
AND a.q_define_id IN (SELECT DISTINCT q_define_id FROM sch_q_define WHERE open_url IN ({data_quest_url}))
GROUP BY a.q_quest_id,a.yb_id
) b ON a.yb_id = b.yb_id
WHERE a.is_submit = 1
GROUP BY a.yb_id,a.school_id,a.xm,a.x,a.sfzhm,a.xh,a.bjhm,a.year,if(a.is_male=1,'男','女')
ORDER BY a.xh) a
GROUP BY school_id,year,xh,sfzhm) a
Right JOIN #连学校表count一个学校的学生数量
(SELECT school_id,graduate_year,szyx,zy,xh,sfzh,xl,COUNT(*) AS zyrs FROM sch_school_student WHERE is_delete = 0  AND school_id IN ({data_school_id})
GROUP BY school_id,graduate_year,szyx,xh,sfzh,xl) c ON a.school_id = c.school_id AND a.year = c.graduate_year AND a.xh = c.xh AND UPPER(a.sfzhm) = UPPER(right(c.sfzh,6))
LEFT JOIN (SELECT school_name,school_id FROM sch_school) b ON c.school_id = b.school_id #连对应学校名字
-- where (b.school_name not in ({data_kaichu_school})) or (c.xh not in ({data_kaichu_xh})) or (c.graduate_year not in ({data_kaichu_graduate_year}))
GROUP BY school_name,graduate_year,xl,szyx,c.zy WITH ROLLUP
HAVING school_name IS NOT NULL AND graduate_year IS NOT NULL AND graduate_year in ({year_list})
ORDER BY school_name,graduate_year,xl,szyx,c.zy DESC"""

data = pd.read_sql(con=conn1,sql=sql).fillna('-')

sql=f'''
SELECT school_name as '学校名',graduate_year,xl as '学历',szyx as '学院',zy as '专业',szbj as '班级', zyrs as '姓名',c.xh as '学号',email
FROM #每一个学生的信息，答了问卷的有txrs
(SELECT school_id,year,xh,sfzhm,count(yb_id) AS txrs ,a.xm #txrs用于sum计数
FROM (SELECT a.yb_id,a.school_id,a.xm,a.x AS yx,a.sfzhm,a.xh,a.bjhm AS bj,a.year,if(a.is_male=1,'男','女') AS xb #根据样本id拿到回答的学生信息
FROM 
sch_survey_yb a 
INNER JOIN
(SELECT a.q_quest_id,a.yb_id #根据q_define_id拿到所有样本id
FROM `sch_survey_result` a
WHERE a.is_yb = 1
AND a.q_define_id IN (SELECT DISTINCT q_define_id FROM sch_q_define WHERE open_url IN ({data_quest_url}))
GROUP BY a.q_quest_id,a.yb_id
) b ON a.yb_id = b.yb_id
WHERE a.is_submit = 1
GROUP BY a.yb_id,a.school_id,a.xm,a.x,a.sfzhm,a.xh,a.bjhm,a.year,if(a.is_male=1,'男','女')
ORDER BY a.xh) a
GROUP BY school_id,year,xh,sfzhm,a.xm) a
Right JOIN #连学校表count一个学校的学生数量
(SELECT school_id,graduate_year,szyx,zy,xh,sfzh,szbj,xl,email,xm AS zyrs FROM sch_school_student WHERE is_delete = 0  AND school_id IN ({data_school_id}) GROUP BY school_id,graduate_year,szyx,xh,sfzh,xl) c ON a.school_id = c.school_id AND a.year = c.graduate_year AND a.xh = c.xh AND UPPER(a.sfzhm) = UPPER(right(c.sfzh,6))
LEFT JOIN (SELECT school_name,school_id FROM sch_school) b ON c.school_id = b.school_id #连对应学校名字
where school_name IN ({data_school_name}) IS NOT NULL AND graduate_year in ({year_list}) AND txrs IS NULL
AND ((b.school_name not in ({data_kaichu_school})) or (c.xh not in ({data_kaichu_xh})) or (c.graduate_year not in ({data_kaichu_graduate_year})))
ORDER BY school_name,graduate_year,xl,szyx,c.zy,zyrs,c.xh DESC
'''

data2 = pd.read_sql(con=conn1,sql=sql)

In [8]:
for i in range(len(data_student_survey)):

    #数据准备
    query_name = data_student_survey.loc[i]['name']
    school_name=data_student_survey.loc[i]['school_name']
    year =[int(x) for x in data_student_survey.loc[i]['for_graduate_year'].split(',')]
    if not data_company_survey.empty:
        query_name1 = data_company_survey[data_company_survey['school_name']==school_name]['name'].tolist()[0]
        company_query_num=data_company_num[data_company_num['school_name']==school_name]['num'].values
    else:
        query_name1 = ''
        company_query_num=''

    school_data = data[(data['学校名称']==school_name)&(data['graduate_year'].isin(year))][['xl','院系','专业','总人数','填写人数','回收率','未填写人数']]
    data_no_students=data2[(data2['学校名']==school_name)&(data2['graduate_year'].isin(year))][['学历','学院','专业','班级','姓名','学号']]
    data_no_students.to_excel(f'【{query_name}】调研进度汇报{time_str}.xlsx')
    wb = load_workbook(filename=f'【{query_name}】调研进度汇报{time_str}.xlsx')
    sheet2 = wb.active
    sheet2.title = '未提交明细'
    sheet2['A1']='序号'
    sheet1 = wb.create_sheet('统计',0)
    #导出未填写的邮箱
    data_email = data2[(data2['学校名']==school_name)&(data2['graduate_year'].isin(year))][['姓名','email']]
    null_index = data_email.query('email.isnull()').index
    if len(null_index)==0:
        e_group=[]
        for email in data_email['email'].tolist():
            if not re.findall('@',email):continue
            if 'qq' in email or 'QQ' in email:
                if re.findall('\d+@',email):
                    e_group.append(re.findall('\d+@',email)[0]+'qq.com')
            else:
                e_group.append(email)
        pd.DataFrame(e_group).to_csv(f'【{school_name}】邮箱完整{time_str}.txt',sep='\n',index=False, header = False)
    #第二张表
    make_detail_table(data_no_students,sheet2)
    #第一张表
    make_sum_table(True,school_data,sheet1,query_name,query_name1,company_query_num,curr_time)
    
    wb.save(f'【{query_name}】调研进度汇报{time_str}.xlsx')
    print(f'【{query_name}】调研进度汇报{time_str}.xlsx---成功导出')
print('over')

【郑州电力高等专科学校2023届毕业生就业质量调研】调研进度汇报20230724.xlsx---成功导出
【三峡大学土木与建筑学院2018届毕业生中期培养质量跟踪调研问卷】调研进度汇报20230724.xlsx---成功导出
over
